PROGAMACIÓN DE LA LÓGICA PROGRAMABLE E IMPORTACIÓN DE LIBRERÍAS

In [44]:
from pynq import Overlay
base = Overlay("/home/xilinx/jupyter_notebooks/MisCosas/Config/ADCPlantaNivel.bit")

from pynq.lib import AxiGPIO

import matplotlib.pyplot as plt
import time

DECLARACIÓN DE CONSTANTES

In [74]:
Nbits=16
ParteEntera=9
Escalon=0
Tiempo=90

FUNCIONES PARA EL TRATAMIENTO DE LOS NÚMEROS

In [46]:
def ToBinary(IN):
    B=[]
    while(IN>1):
        B.append(IN%2)
        IN=IN//2
    B.append(IN)
        
    while(len(B)<Nbits):
        B.append(0)
        
    return B

In [47]:
def ToDecimalBinary(IN):
    B=[]
    for i in range(0, ParteEntera):
        PM=IN-2**(-i-1)
        if(PM>=0): 
            B.append(1)
            IN=PM
        else: B.append(0)
            
    return B

In [48]:
def InvertBinary(IN):
    B=[]
    for i in range(len(IN)-1, -1, -1):
        B.append(IN[i])
        
    return B

In [49]:
def ToDecimalEntero(IN):
    DE=0
    for i in range(len(IN)-1, -1, -1):
        DE+=(2**i)*IN[i]
        
    return DE

In [50]:
def ToDecimalFraccional(IN):
    DF=0
    for i in range(len(IN)-1, -1, -1):
        DF+=(2**(-i-1))*IN[i]
        
    return DF

In [51]:
#Aparece esta nueva función, encargada de convertir el número digital proporcionado por 
#el XADC al valor analógico original.
def ValorAnalogico(IN):
    B=[]
    B=ToBinary(IN);
    #Se descartan 4-bits porque la conversión es de 12-bits.
    VA=ToDecimalEntero(B[4:17])
    #4095=2^(12)-1 
    VA=VA/4095
    
    return VA

ACCESO A LOS BLOQUES AXI GPIO

In [52]:
BloqPlanta=base.ip_dict['axi_gpio_0']
PV=AxiGPIO(BloqPlanta).channel1
CanalDeSalida=AxiGPIO(BloqPlanta).channel2
SP=CanalDeSalida[0:16]
ResetAvisador=CanalDeSalida[16:17]

BloqADC=base.ip_dict['axi_gpio_1']
Aviso=AxiGPIO(BloqADC).channel1
XADC=AxiGPIO(BloqADC).channel2

# PROGRAMA

SIMULACIÓN CON GRÁFICA A TIEMPO NO REAL

In [79]:
Escalon=0
BE=InvertBinary(ToBinary(int(Escalon//1))[0:Nbits-ParteEntera])+ToDecimalBinary(Escalon-Escalon//1)
SP.write(ToDecimalEntero(InvertBinary(BE)))

In [80]:
x, y, t=[]

for i in range(Tiempo):
    ResetAvisador.write(1)
    
    #Obtenemos el valor analógico presente en el XADC ya en el rango [0, 100] 
    Escalon=ValorAnalogico(XADC.read())*100
    
    BE=InvertBinary(ToBinary(int(Escalon//1))[0:Nbits-ParteEntera])+ToDecimalBinary(Escalon-Escalon//1)
    SP.write(ToDecimalEntero(InvertBinary(BE)))
    
    t.append(i)
    x.append(Escalon)
    y.append(PV.read())
    
    #Como la señal Aviso tiene un oeríodo de 1s, no es necesario realizar un time.sleep(1), tan solo esperar
    #a que esta señal esté a '1'.
    while(Aviso.read()!=1):
        time.sleep(0.05)
        
    ResetAvisador.write(0)
    
Input=x

In [81]:
Output=[]
for i in range(len(y)):
    #Pasamos a binario la lectura de la FPGA.
    S=ToBinary(y[i])
    #Para la parte entera eliminamos los bits decimales, del 0 hasta el 8.
    #Para la parte decimal eliminamos los bits enteros, del 9 hasta el 15.
    Output.append(ToDecimalEntero(S[ParteEntera:Nbits])+ToDecimalFraccional(InvertBinary(S[0:ParteEntera])))

In [ ]:
fig, P1= plt.subplots() 
P1.plot(t, x, "b", label="CV")
P1.plot(t, Output, "r", label="PV")
plt.xlim(0, Tiempo)
plt.ylim(0, 100)
plt.legend(loc = "best")
plt.title('Planta de control de nivel')
plt.xlabel("Tiempo (s)")
plt.ylabel("Nivel (%)");